## importing libraries

Create a dbconfig.py file in the current directory like this to connect to database:<br>
user= ---- <br>
password= ---- <br>
host= ---- <br>
database= ---- <br>

In [1]:
import pandas as pd
import mysql.connector as sql
from dbconfig import user,password,host,database
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from flaml import AutoML
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.decomposition import PCA

## Getting data from database

In [2]:
with sql.connect(user= user, password= password, host= host, database= database) as cn:
    cur = cn.cursor()
    
    cur.execute("""
                SELECT `order`.`Order ID`, `Customer ID`, `Order Priority`, `Order Date`, 
                        `Market`, `Product ID`, `Sales`, `Quantity`, `Discount`, `Profit`, `Shipping Cost` 
                FROM order_detail
                JOIN `order`
                ON order_detail.`Order ID` = `order`.`Order ID`
                """)
    fact_df = pd.DataFrame(cur.fetchall(), columns= ['Order ID', 'Customer ID', 'Order Priority', 'Order Date', 
                        'Market', 'Product ID', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost'])
    cur.execute("""
                SELECT * FROM superstore.customer;
                """)
    customer_df = pd.DataFrame(cur.fetchall(), columns= ["Customer ID", "Customer Name", "Segment"])
    
    cur.execute("""
                SELECT * FROM superstore.product;
                """)
    product_df = pd.DataFrame(cur.fetchall(), columns= ["Product ID", "Product Name", "Category", "Sub-Category"])
    
    cur.execute("""
                SELECT * FROM superstore.shipping;
                """)
    shipping_df = pd.DataFrame(cur.fetchall(), columns= ["Shipping ID", "Order ID", "Ship Date", "Ship Mode", "City", "State", "Country", "Region"])
    
    cur.execute("""
                SELECT * FROM superstore.returned;
                """)
    return_df = pd.DataFrame(cur.fetchall(), columns= ["Order ID"])

## Checking dataframes

In [3]:
fact_df

,Order ID,Customer ID,Order Priority,Order Date,Market,Product ID,Sales,Quantity,Discount,Profit,Shipping Cost
0,AE-2011-9160,PO-8865,Medium,2011-10-03,EMEA,TEC-EPS-10004171,78.408,6,0.7,-88.992,3.87
1,AE-2011-9160,PO-8865,Medium,2011-10-03,EMEA,OFF-FEL-10001405,82.674,2,0.7,-157.086,5.69
2,AE-2013-1130,EB-4110,High,2013-10-14,EMEA,OFF-ACC-10004278,4.248,1,0.7,-4.692,0.10
3,AE-2013-1130,EB-4110,High,2013-10-14,EMEA,FUR-BUS-10003055,224.748,6,0.7,-232.272,60.08
4,AE-2013-1530,MY-7380,High,2013-12-31,EMEA,OFF-TEN-10002817,6.966,1,0.7,-8.604,1.75
...,...,...,...,...,...,...,...,...,...,...,...
49665,ZI-2014-7610,BS-1380,Medium,2014-03-24,Africa,OFF-ADV-10000551,4.077,1,0.7,-9.513,0.09
49666,ZI-2014-9540,BT-1395,Medium,2014-06-09,Africa,OFF-BIN-10001274,7.164,2,0.7,-6.696,0.35
49667,ZI-2014-9550,JC-5775,Medium,2014-12-18,Africa,TEC-KON-10003116,71.640,2,0.7,-93.180,4.05
49668,ZI-2014-9550,JC-5775,Medium,2014-12-18,Africa,OFF-AVE-10000543,1.161,1,0.7,-2.349,0.07


In [4]:
customer_df

,Customer ID,Customer Name,Segment
0,AA-10315,Alex Avila,Consumer
1,AA-10375,Allen Armold,Consumer
2,AA-10480,Andrew Allen,Consumer
3,AA-10645,Anna Andreadi,Consumer
4,AA-315,Alex Avila,Consumer
...,...,...,...
1584,YS-21880,Yana Sorensen,Corporate
1585,ZC-11910,Zuschuss Carroll,Consumer
1586,ZC-21910,Zuschuss Carroll,Consumer
1587,ZD-11925,Zuschuss Donatelli,Consumer


In [5]:
product_df

,Product ID,Product Name,Category,Sub-Category
0,FUR-ADV-10000002,"Advantus Photo Frame, Duo Pack",Furniture,Furnishings
1,FUR-ADV-10000108,"Advantus Clock, Erganomic",Furniture,Furnishings
2,FUR-ADV-10000183,"Advantus Photo Frame, Black",Furniture,Furnishings
3,FUR-ADV-10000188,"Advantus Stacking Tray, Erganomic",Furniture,Furnishings
4,FUR-ADV-10000190,"Advantus Frame, Duo Pack",Furniture,Furnishings
...,...,...,...,...
10241,TEC-STA-10004181,"StarTech Inkjet, Durable",Technology,Machines
10242,TEC-STA-10004536,"StarTech Inkjet, Wireless",Technology,Machines
10243,TEC-STA-10004542,"StarTech Calculator, Durable",Technology,Machines
10244,TEC-STA-10004834,"StarTech Receipt Printer, Red",Technology,Machines


In [6]:
shipping_df

,Shipping ID,Order ID,Ship Date,Ship Mode,City,State,Country,Region
0,21043,CA-2012-124891,2012-07-31,Same Day,New York City,New York,United States,East
1,21044,IN-2013-77878,2013-02-07,Second Class,Wollongong,New South Wales,Australia,Oceania
2,21045,IN-2013-71249,2013-10-18,First Class,Brisbane,Queensland,Australia,Oceania
3,21046,ES-2013-1579342,2013-01-30,First Class,Berlin,Berlin,Germany,Central
4,21047,SG-2013-4320,2013-11-06,Same Day,Dakar,Dakar,Senegal,Africa
...,...,...,...,...,...,...,...,...
25028,46071,ZI-2011-4350,2011-03-26,Standard Class,Harare,Harare,Zimbabwe,Africa
25029,46072,MX-2014-169530,2014-06-11,First Class,Bragança Paulista,São Paulo,Brazil,South
25030,46073,IN-2014-72327,2014-05-30,Same Day,Lucknow,Uttar Pradesh,India,Central Asia
25031,46074,IN-2014-57662,2014-08-10,Standard Class,Townsville,Queensland,Australia,Oceania


In [7]:
return_df

,Order ID
0,CA-2011-100762
1,CA-2011-100867
2,CA-2011-102652
3,CA-2011-103373
4,CA-2011-103744
...,...
1167,US-2014-160745
1168,US-2014-161830
1169,US-2014-167178
1170,US-2014-167500


### Transforming the datasets

converting 'Order ID' column to string and creating a boolean 'return' column in return_df dataframe

In [8]:
return_df["Order ID"] = return_df["Order ID"].astype(str)
return_df["return"] = True

converting 'Order ID' column to string in fact_df dataframe and joining fact_df and return_df so fact_df has a 'return' column showing whether it has returned or not

In [9]:
fact_df["Order ID"] = fact_df["Order ID"].astype(str)
fact_df = fact_df.set_index('Order ID').join(return_df.set_index('Order ID')).fillna(False).reset_index()
fact_df

,Order ID,Customer ID,Order Priority,Order Date,Market,Product ID,Sales,Quantity,Discount,Profit,Shipping Cost,return
0,AE-2011-9160,PO-8865,Medium,2011-10-03,EMEA,TEC-EPS-10004171,78.408,6,0.7,-88.992,3.87,False
1,AE-2011-9160,PO-8865,Medium,2011-10-03,EMEA,OFF-FEL-10001405,82.674,2,0.7,-157.086,5.69,False
2,AE-2013-1130,EB-4110,High,2013-10-14,EMEA,OFF-ACC-10004278,4.248,1,0.7,-4.692,0.10,False
3,AE-2013-1130,EB-4110,High,2013-10-14,EMEA,FUR-BUS-10003055,224.748,6,0.7,-232.272,60.08,False
4,AE-2013-1530,MY-7380,High,2013-12-31,EMEA,OFF-TEN-10002817,6.966,1,0.7,-8.604,1.75,False
...,...,...,...,...,...,...,...,...,...,...,...,...
49665,ZI-2014-7610,BS-1380,Medium,2014-03-24,Africa,OFF-ADV-10000551,4.077,1,0.7,-9.513,0.09,False
49666,ZI-2014-9540,BT-1395,Medium,2014-06-09,Africa,OFF-BIN-10001274,7.164,2,0.7,-6.696,0.35,False
49667,ZI-2014-9550,JC-5775,Medium,2014-12-18,Africa,TEC-KON-10003116,71.640,2,0.7,-93.180,4.05,False
49668,ZI-2014-9550,JC-5775,Medium,2014-12-18,Africa,OFF-AVE-10000543,1.161,1,0.7,-2.349,0.07,False


joining the fact_df, customer_df, shipping_df, and product_df to have a dataset containing all details about all products selled names 'model_data'

In [10]:
model_data = \
    fact_df.set_index('Customer ID').join(customer_df.set_index('Customer ID')).reset_index()\
    .set_index("Order ID")      .join(shipping_df.set_index("Order ID")).reset_index()\
    .set_index("Product ID")    .join(product_df.set_index("Product ID")).reset_index()
model_data

,Product ID,Order ID,Customer ID,Order Priority,Order Date,Market,Sales,Quantity,Discount,Profit,...,Shipping ID,Ship Date,Ship Mode,City,State,Country,Region,Product Name,Category,Sub-Category
0,FUR-ADV-10000002,IZ-2014-4660,TB-11190,Medium,2014-09-08,EMEA,53.040,1,0.0,20.130,...,26681,2014-09-15,Standard Class,Baghdad,Baghdad,Iraq,EMEA,"Advantus Photo Frame, Duo Pack",Furniture,Furnishings
1,FUR-ADV-10000108,LI-2011-9980,PM-9135,High,2011-04-13,Africa,100.020,2,0.0,0.960,...,31205,2011-04-15,Second Class,Monrovia,Montserrado,Liberia,Africa,"Advantus Clock, Erganomic",Furniture,Furnishings
2,FUR-ADV-10000108,MO-2012-8110,AR-540,Medium,2012-05-10,Africa,50.010,1,0.0,0.480,...,41952,2012-05-17,Standard Class,Casablanca,Grand Casablanca,Morocco,Africa,"Advantus Clock, Erganomic",Furniture,Furnishings
3,FUR-ADV-10000108,RW-2012-8200,SG-10890,Medium,2012-02-20,Africa,200.040,4,0.0,1.920,...,35366,2012-02-24,Standard Class,Kigali,Kigali,Rwanda,Africa,"Advantus Clock, Erganomic",Furniture,Furnishings
4,FUR-ADV-10000183,IZ-2014-5670,BP-1155,Medium,2014-01-24,EMEA,52.980,1,0.0,0.510,...,39223,2014-01-27,First Class,Baghdad,Baghdad,Iraq,EMEA,"Advantus Photo Frame, Black",Furniture,Furnishings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49665,TEC-STA-10004542,TU-2014-8160,SC-10260,Low,2014-07-07,EMEA,16.032,1,0.6,-18.048,...,31599,2014-07-13,Standard Class,Istanbul,Istanbul,Turkey,EMEA,"StarTech Calculator, Durable",Technology,Machines
49666,TEC-STA-10004542,UP-2012-110,AB-150,Medium,2012-03-22,EMEA,80.160,2,0.0,12.000,...,35649,2012-03-23,First Class,Kharkiv,Kharkiv,Ukraine,EMEA,"StarTech Calculator, Durable",Technology,Machines
49667,TEC-STA-10004834,NI-2013-7410,CD-1920,Low,2013-08-28,Africa,32.319,1,0.7,-37.731,...,38847,2013-09-03,Standard Class,Kano,Kano,Nigeria,Africa,"StarTech Receipt Printer, Red",Technology,Machines
49668,TEC-STA-10004834,TU-2011-790,CC-2145,Medium,2011-11-03,EMEA,172.368,4,0.6,-107.832,...,34025,2011-11-08,Standard Class,Adana,Adana,Turkey,EMEA,"StarTech Receipt Printer, Red",Technology,Machines


check the columns in model_data

In [11]:
model_data.columns

Index(['Product ID', 'Order ID', 'Customer ID', 'Order Priority', 'Order Date',
       'Market', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost',
       'return', 'Customer Name', 'Segment', 'Shipping ID', 'Ship Date',
       'Ship Mode', 'City', 'State', 'Country', 'Region', 'Product Name',
       'Category', 'Sub-Category'],
      dtype='object')

grouping the dataset by 'Order ID' and saving all values in each column in a pandas array

In [12]:
model_data = model_data.groupby("Order ID").agg(pd.array)
model_data

,Product ID,Customer ID,Order Priority,Order Date,Market,Sales,Quantity,Discount,Profit,Shipping Cost,...,Shipping ID,Ship Date,Ship Mode,City,State,Country,Region,Product Name,Category,Sub-Category
Order ID,,,,,,,,,,,,,,,,,,,,,
AE-2011-9160,"[OFF-FEL-10001405, TEC-EPS-10004171]","[PO-8865, PO-8865]","[Medium, Medium]","[2011-10-03 00:00:00, 2011-10-03 00:00:00]","[EMEA, EMEA]","[82.674, 78.408]","[2, 6]","[0.7, 0.7]","[-157.086, -88.992]","[5.69, 3.87]",...,"[38921, 38921]","[2011-10-07 00:00:00, 2011-10-07 00:00:00]","[Standard Class, Standard Class]","[Ajman, Ajman]","['Ajman, 'Ajman]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]","[Fellowes File Cart, Industrial, Epson Calcula...","[Office Supplies, Technology]","[Storage, Machines]"
AE-2013-1130,"[FUR-BUS-10003055, OFF-ACC-10004278]","[EB-4110, EB-4110]","[High, High]","[2013-10-14 00:00:00, 2013-10-14 00:00:00]","[EMEA, EMEA]","[224.748, 4.248]","[6, 1]","[0.7, 0.7]","[-232.272, -4.692]","[60.08, 0.1]",...,"[25684, 25684]","[2013-10-14 00:00:00, 2013-10-14 00:00:00]","[Same Day, Same Day]","[Ras al Khaymah, Ras al Khaymah]","[Ra's Al Khaymah, Ra's Al Khaymah]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]","[Bush Stackable Bookrack, Pine, Accos Paper Cl...","[Furniture, Office Supplies]","[Bookcases, Fasteners]"
AE-2013-1530,"[OFF-STI-10000114, OFF-TEN-10002817]","[MY-7380, MY-7380]","[High, High]","[2013-12-31 00:00:00, 2013-12-31 00:00:00]","[EMEA, EMEA]","[16.668, 6.966]","[2, 1]","[0.7, 0.7]","[-29.472, -8.604]","[1.41, 1.75]",...,"[43486, 43486]","[2014-01-03 00:00:00, 2014-01-03 00:00:00]","[Second Class, Second Class]","[Ras al Khaymah, Ras al Khaymah]","[Ra's Al Khaymah, Ra's Al Khaymah]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]","[Stiletto Letter Opener, High Speed, Tenex Fol...","[Office Supplies, Office Supplies]","[Supplies, Storage]"
AE-2014-2840,[OFF-ROG-10001340],[PG-8820],[Critical],[2014-11-05 00:00:00],[EMEA],[42.48],[1],[0.7],[-75.06],[8.04],...,[37088],[2014-11-08 00:00:00],[First Class],[Ajman],['Ajman],[United Arab Emirates],[EMEA],"[Rogers File Cart, Industrial]",[Office Supplies],[Storage]
AE-2014-3830,"[OFF-AVE-10000357, OFF-AVE-10004827, OFF-BIC-1...","[GH-4665, GH-4665, GH-4665, GH-4665, GH-4665, ...","[Medium, Medium, Medium, Medium, Medium, Medium]","[2014-12-13 00:00:00, 2014-12-13 00:00:00, 201...","[EMEA, EMEA, EMEA, EMEA, EMEA, EMEA]","[3.159, 3.429, 77.256, 59.373, 42.489, 95.796]","[1, 1, 8, 1, 1, 4]","[0.7, 0.7, 0.7, 0.7, 0.7, 0.7]","[-4.971, -3.891, -82.584, -118.767, -62.331, -...","[0.25, 0.25, 4.08, 5.83, 2.24, 6.73]",...,"[38045, 38045, 38045, 38045, 38045, 38045]","[2014-12-19 00:00:00, 2014-12-19 00:00:00, 201...","[Standard Class, Standard Class, Standard Clas...","[Ras al Khaymah, Ras al Khaymah, Ras al Khayma...","[Ra's Al Khaymah, Ra's Al Khaymah, Ra's Al Kha...","[United Arab Emirates, United Arab Emirates, U...","[EMEA, EMEA, EMEA, EMEA, EMEA, EMEA]","[Avery Binder Covers, Clear, Avery Binder Cove...","[Office Supplies, Office Supplies, Office Supp...","[Binders, Binders, Art, Storage, Storage, Phones]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZI-2014-7160,"[OFF-EAT-10004696, OFF-ROG-10001549, OFF-TEN-1...","[EH-4125, EH-4125, EH-4125]","[Medium, Medium, Medium]","[2014-10-30 00:00:00, 2014-10-30 00:00:00, 201...","[Africa, Africa, Africa]","[15.894, 18.639, 16.542]","[2, 1, 1]","[0.7, 0.7, 0.7]","[-27.606, -40.401, -28.128]","[0.19, 3.12, 0.41]",...,"[41595, 41595, 41595]","[2014-11-01 00:00:00, 2014-11-01 00:00:00, 201...","[First Class, First Class, First Class]","[Chitungwiza, Chitungwiza, Chitungwiza]","[Harare, Harare, Harare]","[Zimbabwe, Zimbabwe, Zimbabwe]","[Africa, Africa, Africa]","[Eaton Note Cards, 8.5 x 11, Rogers Shelving, ...","[Office Supplies, Office Supplies, Office Supp...","[Paper, Storage, Storage]"
ZI-2014-7610,"[OFF-ADV-10000551, TEC-STA-10000699]","[BS-1380, BS-1380]","[Medium, Medium]","[2014-03-24 0

Dropping Unneccessary columns

In [13]:
model_data.drop(labels= ['Product ID', 'Customer ID', 'Customer Name', 'Shipping ID',\
                        'City', 'State', 'Product Name', 'Category', 'Sub-Category'], axis= 1, inplace= True)
model_data

,Order Priority,Order Date,Market,Sales,Quantity,Discount,Profit,Shipping Cost,return,Segment,Ship Date,Ship Mode,Country,Region
Order ID,,,,,,,,,,,,,,
AE-2011-9160,"[Medium, Medium]","[2011-10-03 00:00:00, 2011-10-03 00:00:00]","[EMEA, EMEA]","[82.674, 78.408]","[2, 6]","[0.7, 0.7]","[-157.086, -88.992]","[5.69, 3.87]","[False, False]","[Consumer, Consumer]","[2011-10-07 00:00:00, 2011-10-07 00:00:00]","[Standard Class, Standard Class]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]"
AE-2013-1130,"[High, High]","[2013-10-14 00:00:00, 2013-10-14 00:00:00]","[EMEA, EMEA]","[224.748, 4.248]","[6, 1]","[0.7, 0.7]","[-232.272, -4.692]","[60.08, 0.1]","[False, False]","[Consumer, Consumer]","[2013-10-14 00:00:00, 2013-10-14 00:00:00]","[Same Day, Same Day]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]"
AE-2013-1530,"[High, High]","[2013-12-31 00:00:00, 2013-12-31 00:00:00]","[EMEA, EMEA]","[16.668, 6.966]","[2, 1]","[0.7, 0.7]","[-29.472, -8.604]","[1.41, 1.75]","[False, False]","[Corporate, Corporate]","[2014-01-03 00:00:00, 2014-01-03 00:00:00]","[Second Class, Second Class]","[United Arab Emirates, United Arab Emirates]","[EMEA, EMEA]"
AE-2014-2840,[Critical],[2014-11-05 00:00:00],[EMEA],[42.48],[1],[0.7],[-75.06],[8.04],[False],[Consumer],[2014-11-08 00:00:00],[First Class],[United Arab Emirates],[EMEA]
AE-2014-3830,"[Medium, Medium, Medium, Medium, Medium, Medium]","[2014-12-13 00:00:00, 2014-12-13 00:00:00, 201...","[EMEA, EMEA, EMEA, EMEA, EMEA, EMEA]","[3.159, 3.429, 77.256, 59.373, 42.489, 95.796]","[1, 1, 8, 1, 1, 4]","[0.7, 0.7, 0.7, 0.7, 0.7, 0.7]","[-4.971, -3.891, -82.584, -118.767, -62.331, -...","[0.25, 0.25, 4.08, 5.83, 2.24, 6.73]","[False, False, False, False, False, False]","[Consumer, Consumer, Consumer, Consumer, Consu...","[2014-12-19 00:00:00, 2014-12-19 00:00:00, 201...","[Standard Class, Standard Class, Standard Clas...","[United Arab Emirates, United Arab Emirates, U...","[EMEA, EMEA, EMEA, EMEA, EMEA, EMEA]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZI-2014-7160,"[Medium, Medium, Medium]","[2014-10-30 00:00:00, 2014-10-30 00:00:00, 201...","[Africa, Africa, Africa]","[15.894, 18.639, 16.542]","[2, 1, 1]","[0.7, 0.7, 0.7]","[-27.606, -40.401, -28.128]","[0.19, 3.12, 0.41]","[False, False, False]","[Home Office, Home Office, Home Office]","[2014-11-01 00:00:00, 2014-11-01 00:00:00, 201...","[First Class, First Class, First Class]","[Zimbabwe, Zimbabwe, Zimbabwe]","[Africa, Africa, Africa]"
ZI-2014-7610,"[Medium, Medium]","[2014-03-24 00:00:00, 2014-03-24 00:00:00]","[Africa, Africa]","[4.077, 21.501]","[1, 1]","[0.7, 0.7]","[-9.513, -20.799]","[0.09, 2.25]","[False, False]","[Corporate, Corporate]","[2014-03-28 00:00:00, 2014-03-28 00:00:00]","[Standard Class, Standard Class]","[Zimbabwe, Zimbabwe]","[Africa, Africa]"
ZI-2014-9540,[Medium],[2014-06-09 00:00:00],[Africa],[7.164],[2],[0.7],[-6.696],[0.35],[False],[Corporate],[2014-06-15 00:00:00],[Standard Class],[Zimbabwe],[Africa]


Extracting unique values in columns as a object outside an array and summing the prices and calculating original price for discounted sales

In [14]:
column_names_with_one_value = ['Order Date', 'Order Priority', 'Market', 'return', 'Segment', 'Ship Mode', 'Ship Date', 'Country', 'Region']
for col_name in column_names_with_one_value:
    model_data[col_name] = model_data[col_name].apply(lambda x: x[0])

model_data['Shipping Cost'] = model_data['Shipping Cost'].apply(sum)
model_data['paidPrice'] = model_data['Sales'].apply(sum)

model_data['OriginalPrices'] = model_data['Sales'] / (1 - model_data['Discount'])
model_data['OriginalPrice'] = model_data['OriginalPrices'].apply(sum)

model_data['Profit'] = model_data['Profit'].apply(sum)

model_data['Quantity'] = model_data['Quantity'].apply(sum)

model_data['Aggregated Discount'] = 1 - (model_data['paidPrice'] / model_data['OriginalPrice'])

model_data.drop(['Sales', 'Discount', 'OriginalPrices'], axis=1, inplace= True)
model_data.reset_index(drop= True, inplace= True)
model_data

,Order Priority,Order Date,Market,Quantity,Profit,Shipping Cost,return,Segment,Ship Date,Ship Mode,Country,Region,paidPrice,OriginalPrice,Aggregated Discount
0,Medium,2011-10-03,EMEA,8,-246.078,9.56,False,Consumer,2011-10-07,Standard Class,United Arab Emirates,EMEA,161.082,536.94,0.7
1,High,2013-10-14,EMEA,7,-236.964,60.18,False,Consumer,2013-10-14,Same Day,United Arab Emirates,EMEA,228.996,763.32,0.7
2,High,2013-12-31,EMEA,3,-38.076,3.16,False,Corporate,2014-01-03,Second Class,United Arab Emirates,EMEA,23.634,78.78,0.7
3,Critical,2014-11-05,EMEA,1,-75.060,8.04,False,Consumer,2014-11-08,First Class,United Arab Emirates,EMEA,42.480,141.60,0.7
4,Medium,2014-12-13,EMEA,16,-429.108,19.38,False,Consumer,2014-12-19,Standard Class,United Arab Emirates,EMEA,281.502,938.34,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25028,Medium,2014-10-30,Africa,4,-96.135,3.72,False,Home Office,2014-11-01,First Class,Zimbabwe,Africa,51.075,170.25,0.7
25029,Medium,2014-03-24,Africa,2,-30.312,2.34,False,Corporate,2014-03-28,Standard Class,Zimbabwe,Africa,25.578,85.26,0.7
25030,Medium,2014-06-09,Africa,2,-6.696,0.35,False,Corporate,2014-06-15,Standard Class,Zimbabwe,Africa,7.164,23.88,0.7
25031,Medium,2014-12-18,Africa,3,-95.529,4.12,False,Consumer,2014-12-23,Standard Class,Zimbabwe,Africa,72.801,242.67,0.7


converting 'Ship Mode' to scalar values

In [15]:
class_dict = {'Standard Class':1, 'Same Day':4, 'Second Class':2, 'First Class':3}
model_data['Ship Mode'] = model_data['Ship Mode'].apply(lambda x: class_dict[x])

converting 'Order Priority' to scalar values

In [16]:
priority_dict = {'Medium':2, 'High':3, 'Critical':4, 'Low':1}
model_data['Order Priority'] = model_data['Order Priority'].apply(lambda x: priority_dict[x])

extracting 'OrderWeekday' and 'ShipWeekday' columns

In [17]:
model_data["OrderWeekday"] = model_data["Order Date"].dt.day_of_week
model_data["ShipWeekday"] = model_data["Ship Date"].dt.day_of_week
model_data.drop(["Order Date", "Ship Date"], axis=1, inplace= True)
model_data

,Order Priority,Market,Quantity,Profit,Shipping Cost,return,Segment,Ship Mode,Country,Region,paidPrice,OriginalPrice,Aggregated Discount,OrderWeekday,ShipWeekday
0,2,EMEA,8,-246.078,9.56,False,Consumer,1,United Arab Emirates,EMEA,161.082,536.94,0.7,0,4
1,3,EMEA,7,-236.964,60.18,False,Consumer,4,United Arab Emirates,EMEA,228.996,763.32,0.7,0,0
2,3,EMEA,3,-38.076,3.16,False,Corporate,2,United Arab Emirates,EMEA,23.634,78.78,0.7,1,4
3,4,EMEA,1,-75.060,8.04,False,Consumer,3,United Arab Emirates,EMEA,42.480,141.60,0.7,2,5
4,2,EMEA,16,-429.108,19.38,False,Consumer,1,United Arab Emirates,EMEA,281.502,938.34,0.7,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25028,2,Africa,4,-96.135,3.72,False,Home Office,3,Zimbabwe,Africa,51.075,170.25,0.7,3,5
25029,2,Africa,2,-30.312,2.34,False,Corporate,1,Zimbabwe,Africa,25.578,85.26,0.7,0,4
25030,2,Africa,2,-6.696,0.35,False,Corporate,1,Zimbabwe,Africa,7.164,23.88,0.7,0,6
25031,2,Africa,3,-95.529,4.12,False,Consumer,1,Zimbabwe,Africa,72.801,242.67,0.7,3,1


One-Hot-encoding market, country, region, order weekday, shipping weekday, and customer segments and dropping the original columns

In [18]:
encoded_market = pd.get_dummies(model_data["Market"], prefix= "mkt_")
encoded_country = pd.get_dummies(model_data["Country"], prefix= "cnt_")
encoded_segment = pd.get_dummies(model_data["Segment"], prefix= "seg_")
encoded_region = pd.get_dummies(model_data["Region"], prefix= "reg_")
encoded_orderweekday = pd.get_dummies(model_data["OrderWeekday"], prefix= "owd_")
encoded_shipweekday = pd.get_dummies(model_data["ShipWeekday"], prefix= "swd_")
model_data.drop(["Market", "Country", "Segment", "Region", "OrderWeekday", "ShipWeekday"], axis=1, inplace= True)
grouped_model_data = pd.concat([model_data,encoded_segment,encoded_country,encoded_market,encoded_region,encoded_orderweekday,encoded_shipweekday], axis=1)
grouped_model_data

,Order Priority,Quantity,Profit,Shipping Cost,return,Ship Mode,paidPrice,OriginalPrice,Aggregated Discount,seg__Consumer,...,owd__4,owd__5,owd__6,swd__0,swd__1,swd__2,swd__3,swd__4,swd__5,swd__6
0,2,8,-246.078,9.56,False,1,161.082,536.94,0.7,True,...,False,False,False,False,False,False,False,True,False,False
1,3,7,-236.964,60.18,False,4,228.996,763.32,0.7,True,...,False,False,False,True,False,False,False,False,False,False
2,3,3,-38.076,3.16,False,2,23.634,78.78,0.7,False,...,False,False,False,False,False,False,False,True,False,False
3,4,1,-75.060,8.04,False,3,42.480,141.60,0.7,True,...,False,False,False,False,False,False,False,False,True,False
4,2,16,-429.108,19.38,False,1,281.502,938.34,0.7,True,...,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25028,2,4,-96.135,3.72,False,3,51.075,170.25,0.7,False,...,False,False,False,False,False,False,False,False,True,False
25029,2,2,-30.312,2.34,False,1,25.578,85.26,0.7,False,...,False,False,False,False,False,False,False,True,False,False
25030,2,2,-6.696,0.35,False,1,7.164,23.88,0.7,False,...,False,False,False,False,False,False,False,False,False,True
25031,2,3,-95.529,4.12,False,1,72.801,242.67,0.7,True,...,False,False,False,False,True,False,False,False,False,False


Standard scaling and normalizing values

In [19]:
scaler = StandardScaler()
standardizer = MinMaxScaler()
columns_to_transform = grouped_model_data.columns.tolist()   #['Order Priority', 'Quantity', 'Profit', 'Shipping Cost', 'Ship Mode', 'paidPrice', 'OriginalPrice', 'OrderWeekday', 'ShipWeekday']
columns_to_transform.remove("return")
grouped_model_data[columns_to_transform] = scaler.fit_transform(grouped_model_data[columns_to_transform])
grouped_model_data[columns_to_transform] = standardizer.fit_transform(grouped_model_data[columns_to_transform])
grouped_model_data

,Order Priority,Quantity,Profit,Shipping Cost,return,Ship Mode,paidPrice,OriginalPrice,Aggregated Discount,seg__Consumer,...,owd__4,owd__5,owd__6,swd__0,swd__1,swd__2,swd__3,swd__4,swd__5,swd__6
0,0.333333,0.122807,0.424554,0.004603,False,0.000000,0.006785,0.011345,0.823529,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.666667,0.105263,0.425136,0.028979,False,1.000000,0.009655,0.016144,0.823529,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.666667,0.035088,0.437841,0.001521,False,0.333333,0.000975,0.001632,0.823529,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.000000,0.000000,0.435479,0.003871,False,0.666667,0.001772,0.002964,0.823529,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.333333,0.263158,0.412863,0.009332,False,0.000000,0.011874,0.019855,0.823529,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25028,0.333333,0.052632,0.434132,0.001790,False,0.666667,0.002135,0.003571,0.823529,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
25029,0.333333,0.017544,0.438337,0.001126,False,0.000000,0.001058,0.001769,0.823529,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
25030,0.333333,0.017544,0.439846,0.000168,False,0.000000,0.000279,0.000468,0.823529,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25031,0.333333,0.035088,0.434171,0.001983,False,0.000000,0.003053,0.005106,0.823529,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


convertinng return column to integer

In [20]:
grouped_model_data["return"] = grouped_model_data["return"].astype("Int64")

Oversampling the returned orders category using SMOTE algorithm

In [21]:
class_counts = grouped_model_data["return"].value_counts()
minority_class = class_counts.idxmin()
oversampler = SMOTE(sampling_strategy={minority_class: class_counts.max()}, k_neighbors=5)
X_resampled, y_resampled = oversampler.fit_resample(grouped_model_data.drop("return", axis= 1), grouped_model_data["return"])
print('Class counts after oversampling:', y_resampled.value_counts())

Class counts after oversampling: return
0    23861
1    23861
Name: count, dtype: Int64


train-test splitting

In [22]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(grouped_model_data.drop("return", axis =1), grouped_model_data["return"], test_size=0.2, random_state=42)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

## Modelling

In [ ]:
automl = AutoML()
automl_settings = {
    "time_budget": 180, 
    "metric": 'accuracy',
    "task": 'classification',
    "log_file_name": 'returned_orders_automl.log',
    "eval_method": 'cv',
    "n_splits": 5,
    "model_history": True,
}
automl.fit(X_train= X_train,y_train= y_train, **automl_settings)

## Model evaluation

In [25]:
y_pred1 = automl.predict(X= X_test1)
fscore = f1_score(y_test1, y_pred1)
conf_mat = confusion_matrix(y_test1, y_pred1)

In [26]:
print(f"The f1-score for the model is: {round(fscore,4)}\nThe confusion Matrix: \n", conf_mat)
print(f"This means out of {conf_mat[0,0] + conf_mat[1,0] + conf_mat[0,1] + conf_mat[1,1]} orders, the model predicted:\n{conf_mat[0,0]}\tcorrect   not returned\n{conf_mat[1,1]}\tcorrect   returned\n{conf_mat[0,1]}\tincorrect not returned\n{conf_mat[1,0]}\tincorrect returned")

The f1-score for the model is: 0.8571
The confusion Matrix: 
 [[4779    9]
 [  48  171]]
This means out of 5007 orders, the model predicted:
4779	correct   not returned
171	correct   returned
9	incorrect not returned
48	incorrect returned
